In [1]:
# Setup the Jupyter version of Dash
from jupyter_dash import JupyterDash

# Configure the necessary Python module imports for dashboard components
import dash_leaflet as dl
from dash import dcc
from dash import html
import plotly.express as px
from dash import dash_table
from dash.dependencies import Input, Output, State
import base64

# Configure OS routines
import os

# Configure the plotting routines
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

# Import your CRUD Python module and class
from AnimalShelter import AnimalShelter

###########################
# Data Manipulation / Model
###########################
# MongoDB credentials and class instance
username = "aacuser"
password = "SNHU1234"
host = "nv-desktop-services.apporto.com"
db_name = "AAC"

# Initialize the database interface
shelter = AnimalShelter(host, db_name, username, password)

# Retrieve all documents from the database
df = pd.DataFrame.from_records(shelter.read({}))
if '_id' in df.columns:
    df.drop(columns=['_id'], inplace=True)

#########################
# Dashboard Layout / View
#########################
app = JupyterDash(__name__)

# Attempt to load the Grazioso Salvare logo image
image_filename = 'Grazioso Salvare Logo.png'
try:
    encoded_image = base64.b64encode(open(image_filename, 'rb').read())
    logo_img = html.Center(html.Img(src='data:image/png;base64,{}'.format(encoded_image.decode()), height='100px'))
except FileNotFoundError:
    logo_img = html.Center(html.H4("Grazioso Salvare Dashboard"))

# App layout
app.layout = html.Div([
    logo_img,
    html.Center(html.H4("Created by Dominoe LaMattina")),
    html.Center(html.B(html.H1('CS-340 Dashboard'))),
    html.Hr(),

    html.Div([
        html.Label("Filter by Rescue Type"),
        dcc.RadioItems(
            id='filter-type',
            options=[
                {'label': 'Water Rescue', 'value': 'water'},
                {'label': 'Mountain or Wilderness Rescue', 'value': 'mountain'},
                {'label': 'Disaster or Individual Tracking', 'value': 'disaster'},
                {'label': 'Reset', 'value': 'reset'}
            ],
            labelStyle={'display': 'inline-block', 'margin-right': '15px'}
        )
    ], style={'width': '80%', 'margin': 'auto'}),

    html.Hr(),

    dash_table.DataTable(
        id='datatable-id',
        columns=[{"name": i, "id": i, "deletable": False, "selectable": True} for i in df.columns],
        data=df.to_dict('records'),
        row_selectable='single',
        selected_rows=[0],
        filter_action='native',
        sort_action='native',
        page_size=10
    ),

    html.Br(),
    html.Hr(),

    html.Div(
        className='row',
        style={'display': 'flex', 'justify-content': 'space-evenly'},
        children=[
            html.Div(id='graph-id', className='col s12 m6', style={'width': '48%'}),
            html.Div(id='map-id', className='col s12 m6', style={'width': '48%'})
        ]
    )
])

#############################################
# Interaction Between Components / Controller
#############################################

# Query builder for filtering rescue types
def rescue_filter_query(filter_type):
    if filter_type == 'water':
        return {
            'breed': {'$in': ['Labrador Retriever Mix', 'Chesapeake Bay Retriever', 'Newfoundland']},
            'sex_upon_outcome': 'Intact Female',
            'age_upon_outcome_in_weeks': {'$gte': 26, '$lte': 156}
        }
    elif filter_type == 'mountain':
        return {
            'breed': {'$in': ['German Shepherd', 'Alaskan Malamute', 'Old English Sheepdog', 'Siberian Husky', 'Rottweiler']},
            'sex_upon_outcome': 'Intact Male',
            'age_upon_outcome_in_weeks': {'$gte': 26, '$lte': 156}
        }
    elif filter_type == 'disaster':
        return {
            'breed': {'$in': ['Doberman Pinscher', 'German Shepherd', 'Golden Retriever', 'Bloodhound', 'Rottweiler']},
            'sex_upon_outcome': 'Intact Male',
            'age_upon_outcome_in_weeks': {'$gte': 20, '$lte': 300}
        }
    else:
        return {}

# DataTable update callback based on rescue type
@app.callback(Output('datatable-id', 'data'),
              [Input('filter-type', 'value')])
def update_dashboard(filter_type):
    query = rescue_filter_query(filter_type)
    filtered_df = pd.DataFrame.from_records(shelter.read(query))
    if '_id' in filtered_df.columns:
        filtered_df.drop(columns=['_id'], inplace=True)
    return filtered_df.to_dict('records')

# Breed distribution pie chart
@app.callback(
    Output('graph-id', "children"),
    [Input('datatable-id', "derived_virtual_data")])
def update_graphs(viewData):
    if viewData is None:
        return []
    dff = pd.DataFrame.from_dict(viewData)
    return [
        dcc.Graph(
            figure=px.pie(dff, names='breed', title='Breed Distribution')
        )
    ]

# Highlight selected columns in table
@app.callback(
    Output('datatable-id', 'style_data_conditional'),
    [Input('datatable-id', 'selected_columns')]
)
def update_styles(selected_columns):
    if not selected_columns:
        return []
    return [{
        'if': {'column_id': i},
        'background_color': '#D2F3FF'
    } for i in selected_columns]

# Update map based on selected row in data table
@app.callback(
    Output('map-id', "children"),
    [Input('datatable-id', "derived_virtual_data"),
     Input('datatable-id', "derived_virtual_selected_rows")])
def update_map(viewData, index):
    if viewData is None or index is None or len(index) == 0:
        return []
    dff = pd.DataFrame.from_dict(viewData)
    row = index[0]
    return [
        dl.Map(style={'width': '100%', 'height': '500px'}, center=[30.75, -97.48], zoom=10, children=[
            dl.TileLayer(id="base-layer-id"),
            dl.Marker(position=[dff.iloc[row]['location_lat'], dff.iloc[row]['location_long']], children=[
                dl.Tooltip(dff.iloc[row]['breed']),
                dl.Popup([
                    html.H1("Animal Name"),
                    html.P(dff.iloc[row]['name'])
                ])
            ])
        ])
    ]

# Launch the app
app.run_server(debug=True)

Dash app running on http://127.0.0.1:17941/
